In [ ]:
# %% 

import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'    

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import perf_funcs as perf
import hickle as hkl
from numba import njit
from functools import partial
import time
import mat73
from scipy.stats import loguniform, wasserstein_distance
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.preprocessing import KernelCenterer, StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

/Users/Hannah/Documents/VSCode/Volterra


In [ ]:
# %%

#################### Scorers #########################
######################################################

def neg_calculate_nmse(y_true, y_pred):
    # Calculate MSE
    mse = np.mean((y_true - y_pred)**2, axis=0)
    factor = np.mean((y_true)**2, axis=0)
    neg_nmse = -np.mean(mse / factor)
    return neg_nmse

def mean_wasserstein_dist(y_true, y_pred):
    was_dist=[]
    ndim=np.shape(y_true)[1]
    for i in range(ndim):
        was_dist = np.append(was_dist, wasserstein_distance(y_true[:,i], y_pred[:,i]))
    return np.mean(was_dist)

def neg_calculate_mse(y_true, y_pred):
    # Calculate MSE
    neg_mse = -np.mean((y_true - y_pred)**2)
    return neg_mse

def set_scorer(scorer='None'):
    if scorer=='neg_mse':
        my_scorer = {'neg_mse': make_scorer(neg_calculate_mse, greater_is_better=True)}  
        my_refit = 'neg_mse' 
        score_str = 'mean_test_neg_mse'
    elif scorer=='neg_nmse':
        my_scorer = {'neg_nmse': make_scorer(neg_calculate_nmse, greater_is_better=True)}  
        my_refit = 'neg_nmse' 
        score_str = 'mean_test_neg_nmse'
    elif scorer=='score':
        my_scorer = None
        my_refit = True
        score_str = 'mean_test_score'
    return my_scorer, my_refit, score_str

In [ ]:
# %% 

################ Volterra Class ######################
######################################################

@njit
def _volt_gram_fast_njit(X, Y, omega, tau):
    nT_X, nT_Y = X.shape[0], Y.shape[0]
    # Compute once only instead of in the loop
    omega, tau = omega ** 2, tau ** 2
    # Initialize the gram matrix with ones
    Gram = np.ones((nT_X, nT_Y))
    tau_XY = 1 - X @ Y.T * tau
    
    # Compute the first row and column of the Gram matrix
    Gram[0, 0] += omega / (1 - omega) / tau_XY[0, 0]
    for i in range(1, nT_X):
        Gram[i, 0] += omega / (1 - omega) / tau_XY[i, 0]
    for i in range(1, nT_Y):
        Gram[0, i] += omega / (1 - omega) / tau_XY[0, i]

    # Compute the rest of the Gram matrix
    for i in range(1, nT_X):
        tau_XY_i = tau_XY[i]
        for j in range(1, nT_Y):
            Gram[i, j] += omega * Gram[i-1, j-1] / tau_XY_i[j]
    
    return Gram

def _volt_gram(X, Y, omega, tau):
    nT_X = np.shape(X)[0]
    nT_Y = np.shape(Y)[0]
    Gram = np.zeros((nT_X, nT_Y))
    Gram0 = 1/(1-omega**2)
    for i in range(nT_X):
        for j in range(nT_Y):
            if i==0 or j==0:
                Gram[i, j] = 1 + omega**2 * Gram0/(1-(tau**2)*(np.dot(X[i,:], Y[j,:])))
            else:
                Gram[i, j] = 1 + omega**2 * Gram[i-1,j-1]/(1-(tau**2)*(np.dot(X[i,:], Y[j,:])))
    return Gram

def Volterra_kernel_callable(X, Y, omega, tau, nwashout):
    # print(tau)
    # print(omega)
    volt_K = _volt_gram_fast_njit(X, Y, omega, tau)
    return volt_K

class VolterraKernel(BaseEstimator, RegressorMixin):
    def __init__(self, alpha=1, omega=0.1, tau=0.1, nwashout=0, ifscale_kernel=False, ifscale_y=False, ifcenter_kernel=False):
        self.alpha = alpha
        self.omega = omega
        self.tau = tau
        self.nwashout = nwashout
        self.Volterra_kernel = partial(Volterra_kernel_callable, omega=self.omega, tau=self.tau, nwashout=self.nwashout)
        self.regressor = KernelRidge(kernel="precomputed", alpha=self.alpha)
        self.ifscale_kernel = ifscale_kernel
        self.ifscale_y = ifscale_y
        self.ifcenter_kernel = ifcenter_kernel

    def fit(self, X, y):
        # Transform to obtain Volterra kernel and fit kernel ridge regression
        K_fit = self.Volterra_kernel(X, X)
        K_fit = K_fit[self.nwashout:, self.nwashout:]
        if self.ifcenter_kernel:
            centerer = KernelCenterer()
            K_fit = centerer.fit_transform(K_fit)
            self.centerer = centerer
            
        self.X_fit_ = X.copy()
        # Rescale kernel if needed
        if self.ifscale_kernel:
            self.scaler_K = StandardScaler().fit(K_fit)
            K_fit = self.scaler_K.transform(K_fit)

        if self.ifscale_y:
            self.scaler_y = StandardScaler().fit(y)
            y = self.scaler_y.transform(y)

        self.K_fit_ = K_fit
        #print(self.Volterra_kernel)
        self.regressor.fit(self.K_fit_, y[self.nwashout:,:])
        return self
    
    def predict(self, X):
        # Transform to obtain Volterra kernel and predict
        K_test = self.Volterra_kernel(X, self.X_fit_) # Use X_fit, not K_fit
        print(K_test)
        K_test = K_test[:, self.nwashout:]
        if self.ifcenter_kernel:
            centerer = self.centerer
            K_test = centerer.transform(K_test)
            
        if self.ifscale_kernel:
            K_test = self.scaler_K.transform(K_test)

        if self.ifscale_y:
            return self.scaler_y.inverse_transform(self.regressor.predict(K_test))
            
        return self.regressor.predict(K_test)
    
    # Make alpha parameter 'cv-able'
    def set_params(self, **params):
        for k, v in params.items():
            if k == "nwashout":
                self.nwashout = v
            if k == "omega":
                self.omega = v
            elif k == "tau":
                self.tau = v
            elif k == "alpha":
                self.alpha=v
                self.regressor.alpha = self.alpha    
            else:
                setattr(self.regressor, k, v)
        if self.omega > np.sqrt(1 - (self.tau**2)):
            print(self.tau)
            print(self.omega)
            self.omega = np.sqrt(1 - (self.tau**2))*0.99
            print('became:')
            print(self.omega)
        self.Volterra_kernel = partial(Volterra_kernel_callable, 
                                       omega=self.omega, 
                                       tau=self.tau, 
                                       nwashout=self.nwashout)
        self.regressor = KernelRidge(kernel="precomputed", alpha = self.alpha)
        #print(self.Volterra_kernel)
        
        return self

In [ ]:
# %% 

################ Load data ###########################
######################################################

matstruct_contents = mat73.loadmat("./datagen/BEKK_d15_data.mat")
returns = matstruct_contents['data_sim']
epsilons = matstruct_contents['exact_epsilons']
Ht_sim_vech = matstruct_contents['Ht_sim_vech']

data_in = epsilons
#data_out = returns
data_out = Ht_sim_vech

ndim = data_in.shape[1]
nT = data_in.shape[0]
ndim_output = data_out.shape[1]

if nT>3760:
    nT = 3760

x = data_in[0:nT-1,:]
#y = data_out[0:nT-1,:]**2*1000
y = data_out[1:nT,0:ndim_output]*1000
t_dispay = 300

# train test split
x_train, x_test, y_train, y_test = train_test_split(
    x, y, 
    test_size=0.2, 
    shuffle=False, 
    random_state=1)

scaler_y = StandardScaler().fit(y_train)
y_train_demeaned = scaler_y.transform(y_train)

train_len = np.shape(x_train)[0]
test_len = np.shape(x_test)[0]
total_len = test_len + train_len

M = np.max([np.linalg.norm(z) for z in x])
x_train /= M
x_test /= M

set_my_scorer="score"
my_scorer, my_refit, score_str = set_scorer(set_my_scorer)
     
n_jobs = 100

ifsave=True
ifRandomSearch=True

In [ ]:
# %% 

# Check that the Volterra kernel gives the same results on the testing set for some set of hyperparameters
# With modified Gram matrix using my Gram matrix code
test_volt_kernel1 = VolterraKernel()
test_volt_kernel1.fit(x_train, y_train_demeaned)
test_prediction1 = test_volt_kernel1.predict(x_test)

[[1.0100958  1.01010335 1.01009017 ... 1.01010279 1.0100976  1.01009602]
 [1.01009682 1.01008837 1.01010687 ... 1.01010966 1.01009669 1.01009376]
 [1.01010407 1.0101074  1.0100908  ... 1.01009304 1.01010266 1.01010901]
 ...
 [1.01010248 1.01009254 1.01009852 ... 1.01010434 1.01010139 1.01010108]
 [1.01009558 1.01009567 1.01009431 ... 1.01009122 1.01010089 1.01008567]
 [1.0101043  1.01009161 1.01008828 ... 1.01007564 1.0100994  1.01010407]]


In [ ]:
# %% 

# Check for my Volterra kernel prediction set
from estimators.volt_funcs import Volterra
test_volt_kernel2 = Volterra(ld_coef=0.1, tau_coef=0.1, reg=1, washout=0)   # Follow the same defaults
test_volt_kernel2.Train(x_train, y_train_demeaned)
test_prediction2 = test_volt_kernel2.Forecast(x_test)

In [ ]:
# %% 

print(np.allclose(test_prediction1, test_prediction2))
print(test_prediction1)
print(test_prediction2)

True
[[ 8.41487801e-05  1.51256099e-05  1.29824775e-04 ...  5.98599672e-05
  -2.84523764e-04  4.22109810e-04]
 [-1.03571549e-03 -6.85935247e-04 -1.27130554e-03 ... -3.80784750e-04
   7.86217559e-05  4.32546061e-04]
 [-6.48606055e-04 -9.05098567e-05  3.27837266e-04 ... -2.76237920e-04
  -1.12151122e-04 -5.70746494e-04]
 ...
 [-7.09464937e-04 -2.34624089e-04 -5.60455458e-04 ... -3.65696696e-04
  -2.24203087e-04 -3.48580229e-04]
 [-8.58795215e-04 -3.82239735e-04 -6.84032775e-04 ... -3.63015110e-04
  -1.41424250e-05 -2.76493001e-04]
 [-1.73811644e-05 -6.53329903e-05  6.72694680e-04 ...  2.82001292e-04
  -2.47711710e-04 -4.24892451e-04]]
[[ 8.41483681e-05  1.51229414e-05  1.29824565e-04 ...  5.98593685e-05
  -2.84525793e-04  4.22113682e-04]
 [-1.03571590e-03 -6.85937915e-04 -1.27130575e-03 ... -3.80785348e-04
   7.86197274e-05  4.32549933e-04]
 [-6.48606467e-04 -9.05125252e-05  3.27837055e-04 ... -2.76238518e-04
  -1.12153151e-04 -5.70742622e-04]
 ...
 [-7.09465349e-04 -2.34626758e-04 -5.60

In [ ]:
# %% 

# Check to make sure that the Gram matrix is the same

print(test_volt_kernel1.K_fit_)
print(test_volt_kernel2.Gram)

[[1.01012459 1.01010067 1.0101005  ... 1.01010834 1.01010919 1.01010682]
 [1.01010067 1.01012761 1.01010281 ... 1.01010963 1.01010706 1.01009843]
 [1.0101005  1.01010281 1.01015234 ... 1.01009692 1.01010714 1.01009533]
 ...
 [1.01010834 1.01010963 1.01009692 ... 1.01015015 1.01011308 1.01009448]
 [1.01010919 1.01010706 1.01010714 ... 1.01011308 1.01013447 1.01009661]
 [1.01010682 1.01009843 1.01009533 ... 1.01009448 1.01009661 1.01013031]]
[[1.01012459 1.01010067 1.0101005  ... 1.01010834 1.01010919 1.01010682]
 [1.01010067 1.01012761 1.01010281 ... 1.01010963 1.01010706 1.01009843]
 [1.0101005  1.01010281 1.01015234 ... 1.01009692 1.01010714 1.01009533]
 ...
 [1.01010834 1.01010963 1.01009692 ... 1.01015015 1.01011308 1.01009448]
 [1.01010919 1.01010706 1.01010714 ... 1.01011308 1.01013447 1.01009661]
 [1.01010682 1.01009843 1.01009533 ... 1.01009448 1.01009661 1.01013031]]


In [ ]:
# %% 

# Check to make sure that the Gram matrix is the same
print(np.allclose(test_volt_kernel1.K_fit, test_volt_kernel2.Gram))
print(test_volt_kernel1.K_fit_)
print(test_volt_kernel2.Gram)

AttributeError: 'VolterraKernel' object has no attribute 'K_fit'

In [ ]:
# %% 

# Check to make sure that the Gram matrix is the same
print(np.allclose(test_volt_kernel1.K_fit_, test_volt_kernel2.Gram))
print(test_volt_kernel1.K_fit_)
print(test_volt_kernel2.Gram)

True
[[1.01012459 1.01010067 1.0101005  ... 1.01010834 1.01010919 1.01010682]
 [1.01010067 1.01012761 1.01010281 ... 1.01010963 1.01010706 1.01009843]
 [1.0101005  1.01010281 1.01015234 ... 1.01009692 1.01010714 1.01009533]
 ...
 [1.01010834 1.01010963 1.01009692 ... 1.01015015 1.01011308 1.01009448]
 [1.01010919 1.01010706 1.01010714 ... 1.01011308 1.01013447 1.01009661]
 [1.01010682 1.01009843 1.01009533 ... 1.01009448 1.01009661 1.01013031]]
[[1.01012459 1.01010067 1.0101005  ... 1.01010834 1.01010919 1.01010682]
 [1.01010067 1.01012761 1.01010281 ... 1.01010963 1.01010706 1.01009843]
 [1.0101005  1.01010281 1.01015234 ... 1.01009692 1.01010714 1.01009533]
 ...
 [1.01010834 1.01010963 1.01009692 ... 1.01015015 1.01011308 1.01009448]
 [1.01010919 1.01010706 1.01010714 ... 1.01011308 1.01013447 1.01009661]
 [1.01010682 1.01009843 1.01009533 ... 1.01009448 1.01009661 1.01013031]]


In [ ]:
# %% 

# Check that the error functions all run correctly 

from utils.errors import calculate_mse, calculate_nmse, calculate_wasserstein1err

neg_calculate_mse(y_test, test_prediction1)
calculate_mse(y_test, test_prediction2)

0.048006750833792196

In [ ]:
# %% 

# Check that the error functions all run correctly 

from utils.errors import calculate_mse, calculate_nmse, calculate_wasserstein1err

print(neg_calculate_mse(y_test, test_prediction1))
print(calculate_mse(y_test, test_prediction2))

-0.048006750668746746
0.048006750833792196


In [ ]:
# %% 

# Check that the error functions all run correctly 

from utils.errors import calculate_mse, calculate_nmse, calculate_wasserstein1err

print(neg_calculate_mse(y_test, test_prediction1))
print(calculate_mse(y_test, test_prediction2))

print(neg_calculate_nmse(y_test, test_prediction1))
print(calculate_nmse(y_test, test_prediction2))

print(mean_wasserstein_dist(y_test, test_prediction1))
print(calculate_wasserstein1err(y_test, test_prediction2))

-0.048006750668746746
0.048006750833792196
-1.0000104793684867
1.0000104906208724
0.15109413314255987
18.13129606083808


In [ ]:
# %% 

# Check that the error functions all run correctly 

from utils.errors import calculate_mse, calculate_nmse, calculate_wasserstein1err

print(neg_calculate_mse(y_test, test_prediction1))
print(calculate_mse(y_test, test_prediction2))

print(neg_calculate_nmse(y_test, test_prediction1))
print(calculate_nmse(y_test, test_prediction2))

print(mean_wasserstein_dist(y_test, test_prediction1))
print(calculate_wasserstein1err(y_test, test_prediction2))

-0.048006750668746746
0.048006750833792196
-1.0000104793684867
1.0000104906208724
0.15109413314255987
0.1510941338403173
